The PACS maps for HELP have been made by Matt Smith and Kyle Penner. Unfortunetly, the formats differ between the two. For instance Kyle's noise maps are variance rather than standard deviation maps and units differ. This script creates an homogenised format, and attaches other relevant meta data and the ObsIDs.

Steps:
for each field:
* read in coverage, image and noise
* if Kyle's convert units and correct noise
* attach obs IDs
* attach meta data
* create PSF extension

In [1]:
from datetime import datetime
from itertools import product

import numpy as np

from astropy.io import fits
from astropy.table import Table
VERSION = "0.9"

In [8]:
filelist=Table.read('./pacs_maps.csv', format='csv')

In [10]:
all_obsids=Table.read('./pacs_obsid.csv', format='csv')

In [1]:
for i, field in enumerate(np.unique(filelist['field'])):
    #print(filelist[filelist['field']==field])
    
    
    
    obsids = sorted(list(all_obsids['ObsID'][
            all_obsids['field'] == field]))
    
    #loop over band
    for b in [100, 160]:
        
        # to get around Herve's annoying change in number of extensions for cosmos
        if field == 'COSMOS':
            ext=1
        else:
            ext=0
            
            
        # Load in image map
        image_map=fits.open(f"/Volumes/Untitled 2/HELP/hedam.lam.fr/HELP/data/dmu_products/dmu18/dmu18_{field}/data/{filelist[(filelist['field']==field) & (filelist['band']==b) & (filelist['type']=='image')]['file'][0]}")
        image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                data=image_map[ext].data)
        image_hdu.header['EXTNAME'] = "IMAGE"
        #convert unit if required

        if image_hdu.header['BUNIT'] =='Jy/pix':
            image_hdu.data=image_hdu.data*1.0/((3.0462E2)*image_hdu.header['CDELT1']**2)
            image_hdu.header['BUNIT']='MJy/sr'
        
        #to get around the fact Herve didnt bother adding units for COSMOS
        if image_hdu.header['BUNIT'] =='1':
            print('COSMOS now has units')
            image_hdu.header['BUNIT']='MJy/sr'
        
        #Load in noise map
        noise_map=fits.open(f"/Volumes/Untitled 2/HELP/hedam.lam.fr/HELP/data/dmu_products/dmu18/dmu18_{field}/data/{filelist[(filelist['field']==field) & (filelist['band']==b) & (filelist['type']=='noise')]['file'][0]}")
        noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
        noise_hdu.header['EXTNAME'] = "ERROR"
        #convert unit if required
        if noise_hdu.header['BUNIT'] =='Jy/pix':
            noise_hdu.data=noise_hdu.data*1.0/((3.0462E2)*noise_hdu.header['CDELT1']**2)
            noise_hdu.header['BUNIT']='MJy/sr'
                #to get around the fact Herve didnt bother adding units for COSMOS
        if noise_hdu.header['BUNIT'] =='1':
            print('COSMOS now has units')
            noise_hdu.header['BUNIT']='MJy/sr'
        
        #Check if noise is standard deviation by comparing median of noise to standard deviation of image.
        #If not, then correct it
        if np.std(image_hdu.data[np.isfinite(image_hdu.data)]) < np.median(noise_hdu.data[np.isfinite(noise_hdu.data)]):
            noise_hdu.data=np.sqrt(noise_hdu.data)
        
        
        #Load in coverage map. Not all maps have coverage maps so check
        try:
            cov_map=fits.open(f"/Volumes/Untitled 2/HELP/hedam.lam.fr/HELP/data/dmu_products/dmu18/dmu18_{field}/data/{filelist[(filelist['field']==field) & (filelist['band']==b) & (filelist['type']=='cov')]['file'][0]}")
            cov_hdu = fits.ImageHDU(header=cov_map[0].header,
                                data=cov_map[0].data)
            cov_hdu.header['EXTNAME'] = "COVERAGE"
            #convert unit if required
            if cov_hdu.header['BUNIT'] =='Jy/pix':
                cov_hdu.data=cov_hdu.data*1.0/((3.0462E2)*cov_hdu.header['CDELT1']**2)
                cov_hdu.header['BUNIT']='MJy/sr'
        except(IndexError):
            cov_hdu = fits.ImageHDU()
            cov_hdu.header['EXTNAME'] = "COVERAGE"
            cov_hdu.header.add_comment("The coverage map is not available.")
        
         
        
            
        #Create primary header
        primary_hdu = fits.PrimaryHDU()
        primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
        primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
        primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
        ))
        primary_hdu.header.append((
            "VERSION", VERSION, "HELP product version"
        ))
        primary_hdu.header.append((
            "TELESCOP", "Herschel", "Name of the telescope"
        ))
        primary_hdu.header.append((
            "INSTRUME", "PACS", "Name of the instrument"
        ))
        primary_hdu.header.append((
            "FILTER", f"PACS-{str(b)}", "Name of the filter"
        ))
        primary_hdu.header.append((
            "FIELD", field, "Name of the HELP field"
        ))
        for idx, obs_id in enumerate(obsids):
            keyword = "OBSID" + str(idx).zfill(3)
            primary_hdu.header.append((keyword, obs_id))
        
        
        #Add empty extension for PSF
        psf_hdu = fits.ImageHDU()
        psf_hdu.header['EXTNAME'] = "PSF"
        psf_hdu.header.add_comment("The PSF is not available.")
        
        hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
        hdu_list.writeto(f"data/{field}_PACS{str(b)}_v{VERSION}.fits",
                     checksum=True, overwrite=True)
       # print(f"{field} / {b} processed...")


NameError: name 'np' is not defined